# SQL Functions, based `OllamaFunctions`

## Setup

### Imports

In [1]:
import os
import pickle

import db_connect
from test_data import TestData

from langsmith import Client

from langchain_core.language_models import BaseLanguageModel

from langchain.prompts import PromptTemplate

from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits.sql import base
from langchain.agents.agent import AgentExecutor
from langchain.agents import tool

from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain.tools.render import format_tool_to_openai_function

from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

from langchain_core.messages import HumanMessage

### LangSmith

In [2]:
os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Load models

In [3]:
llama3 = OllamaFunctions(model="llama3:8b", temperature=0, format="json")
llama3_inst = OllamaFunctions(model="llama3:instruct", temperature=0, format="json")
llama3_inst_q8 = OllamaFunctions(model="llama3:8b-instruct-q8_0", temperature=0, format="json")
llama3_inst_fp16 = OllamaFunctions(model="llama3:8b-instruct-fp16", temperature=0, format="json")

llama3_text = OllamaFunctions(model="llama3:8b-text", temperature=0, format="json")

### Connect to DB with Readonly role

In [4]:
db = db_connect.get_db()

#### Check connection

In [5]:
db.run("select * from passenger")

"[(1, 'John'), (2, 'James'), (3, 'Poul'), (4, 'Christofer'), (5, 'Superman'), (6, 'Donald'), (7, 'Douglas'), (8, 'Dwight'), (9, 'Earl'), (10, 'Edgar'), (11, 'Edmund'), (12, 'Edwin'), (13, 'Elliot'), (14, 'Eric'), (15, 'Ernest'), (16, 'Ethan'), (17, 'Ezekiel'), (18, 'Felix'), (19, 'Franklin'), (20, 'Frederick'), (21, 'Gabriel'), (22, 'Joseph'), (23, 'Joshua'), (24, 'Julian'), (25, 'Alice'), (26, 'Bob'), (27, 'Charlie'), (28, 'David'), (29, 'Emily'), (30, 'Frank'), (31, 'George'), (32, 'Helen'), (33, 'Irene'), (34, 'Jack'), (35, 'Kate'), (36, 'Leo'), (37, 'Mary'), (38, 'Nancy'), (39, 'Oliver'), (40, 'Paul'), (41, 'Qiana'), (42, 'Robert'), (43, 'Samantha'), (44, 'Thomas'), (45, 'Victoria')]"

-----

## Create agent

In [11]:
@tool
def sql_query_builder():
    """Create a query"""
    return "SELECT * FROM company;"

func = format_tool_to_openai_function(sql_query_builder)
print(func)

{'name': 'sql_query_builder', 'description': 'sql_query_builder() - Create a query', 'parameters': {'type': 'object', 'properties': {}}}


In [15]:
model = llama3_inst.bind_tools([func])

In [17]:
model.invoke("write a joke")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'sql_query_builder', 'arguments': '{"type": "string", "value": "Why did the database go to therapy? Because it was feeling a little \'dis-connected\'!"}'}}, id='run-a691d95a-a6ed-43ee-b1ea-00ac68ed570c-0')